## TF-IDF embedding + similarity

In [15]:
import pandas as pd
import numpy as np

df = pd.read_pickle("data/processed_resumes.pkl")
df_jd = pd.read_pickle("data/processed_jds.pkl")


In [16]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity


In [17]:
tfidf = TfidfVectorizer(
    ngram_range=(1, 2),
    max_features=5000,
    stop_words="english"
)


In [18]:
corpus = (
    df["cleaned_resume"].tolist() +
    df_jd["cleaned_jd"].tolist()
)

tfidf.fit(corpus)


,"input input: {'filename', 'file', 'content'}, default='content'- If `'filename'`, the sequence passed as an argument to fit is expected to be a list of filenames that need reading to fetch the raw content to analyze.- If `'file'`, the sequence items must have a 'read' method (file-like object) that is called to fetch the bytes in memory.- If `'content'`, the input is expected to be a sequence of items that can be of type string or byte.",'content'
,"encoding encoding: str, default='utf-8'If bytes or files are given to analyze, this encoding is used todecode.",'utf-8'
,"decode_error decode_error: {'strict', 'ignore', 'replace'}, default='strict'Instruction on what to do if a byte sequence is given to analyze thatcontains characters not of the given `encoding`. By default, it is'strict', meaning that a UnicodeDecodeError will be raised. Othervalues are 'ignore' and 'replace'.",'strict'
,"strip_accents strip_accents: {'ascii', 'unicode'} or callable, default=NoneRemove accents and perform other character normalizationduring the preprocessing step.'ascii' is a fast method that only works on characters that havea direct ASCII mapping.'unicode' is a slightly slower method that works on any characters.None (default) means no character normalization is performed.Both 'ascii' and 'unicode' use NFKD normalization from:func:`unicodedata.normalize`.",None
,"lowercase lowercase: bool, default=TrueConvert all characters to lowercase before tokenizing.",True
,"preprocessor preprocessor: callable, default=NoneOverride the preprocessing (string transformation) stage whilepreserving the tokenizing and n-grams generation steps.Only applies if ``analyzer`` is not callable.",None
,"tokenizer tokenizer: callable, default=NoneOverride the string tokenization step while preserving thepreprocessing and n-grams generation steps.Only applies if ``analyzer == 'word'``.",None
,"analyzer analyzer: {'word', 'char', 'char_wb'} or callable, default='word'Whether the feature should be made of word or character n-grams.Option 'char_wb' creates character n-grams only from text insideword boundaries; n-grams at the edges of words are padded with space.If a callable is passed it is used to extract the sequence of featuresout of the raw, unprocessed input... versionchanged:: 0.21 Since v0.21, if ``input`` is ``'filename'`` or ``'file'``, the data is first read from the file and then passed to the given callable analyzer.",'word'
,"stop_words stop_words: {'english'}, list, default=NoneIf a string, it is passed to _check_stop_list and the appropriate stoplist is returned. 'english' is currently the only supported stringvalue.There are several known issues with 'english' and you shouldconsider an alternative (see :ref:`stop_words`).If a list, that list is assumed to contain stop words, all of whichwill be removed from the resulting tokens.Only applies if ``analyzer == 'word'``.If None, no stop words will be used. In this case, setting `max_df`to a higher value, such as in the range (0.7, 1.0), can automatically detectand filter stop words based on intra corpus document frequency of terms.",'english'
,"token_pattern token_pattern: str, default=r""(?u)\\b\\w\\w+\\b""Regular expression denoting what constitutes a ""token"", only usedif ``analyzer == 'word'``. The default regexp selects tokens of 2or more alphanumeric characters (punctuation is completely ignoredand always treated as a token separator).If there is a capturing group in token_pattern then thecaptured group content, not the entire match, becomes the token.At most one capturing group is permitted.",'(?u)\\b\\w\\w+\\b'
,"ngram_range ngram_range: tuple (min_n, max_n), default=(1, 1)The lower and upper boundary of the range of n-values for differentn-grams to be extracted. All values of n such that min_n <= n <= max_nwill be used. For example an ``ngram_range`` of ``(1, 1)`` means onlyunigrams, ``(1, 2)`` means unigrams and bigrams, and ``(2, 2)`` meansonly bigrams.Only applies if ``analyzer`` is not callable.

In [19]:
resume_tfidf = tfidf.transform(df["cleaned_resume"])
jd_tfidf = tfidf.transform(df_jd["cleaned_jd"])


In [20]:
jd_idx = 0  # keep consistent

tfidf_scores = cosine_similarity(
    resume_tfidf,
    jd_tfidf[jd_idx]
).flatten()

df["tfidf_similarity"] = tfidf_scores


In [21]:
# saving TF_IDF results
df.to_pickle("data/resumes_with_tfidf.pkl")
print("TF-IDF scores saved")


TF-IDF scores saved


In [22]:
del resume_tfidf
del jd_tfidf


In [23]:
from sentence_transformers import SentenceTransformer

C:\Users\TULSIRAM\AppData\Local\Python\pythoncore-3.14-64\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [24]:
print("Loading SBERT model...")
sbert_model = SentenceTransformer("all-MiniLM-L6-v2")
print("Model loaded")


Loading SBERT model...
Model loaded


In [25]:
resume_texts = df["cleaned_resume"].tolist()
jd_texts = df_jd["cleaned_jd"].tolist()


In [26]:
print("Encoding test batch...")

test_embeddings = sbert_model.encode(
    resume_texts[:5],
    convert_to_numpy=True
)

print("Test embeddings shape:", test_embeddings.shape)


Encoding test batch...
Test embeddings shape: (5, 384)


In [27]:
resume_embeddings = sbert_model.encode(
    resume_texts,
    batch_size=16,
    convert_to_numpy=True,
    show_progress_bar=True
)

jd_embeddings = sbert_model.encode(
    jd_texts,
    batch_size=16,
    convert_to_numpy=True,
    show_progress_bar=True
)


Batches: 100%|██████████| 3/3 [00:01<00:00,  1.67it/s]


In [28]:
np.save("embeddings/resume_embeddings.npy", resume_embeddings)
np.save("embeddings/jd_embeddings.npy", jd_embeddings)

print("Embeddings saved")


Embeddings saved


In [29]:
del resume_embeddings
del jd_embeddings
